# Predicting Salary from job description

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Choosing-the-dependant-and-independant-variables" data-toc-modified-id="Choosing-the-dependant-and-independant-variables-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Choosing the dependant and independant variables</a></span><ul class="toc-item"><li><span><a href="#Independant-variables" data-toc-modified-id="Independant-variables-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Independant variables</a></span><ul class="toc-item"><li><span><a href="#Coding-dummy-variables" data-toc-modified-id="Coding-dummy-variables-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Coding dummy variables</a></span></li></ul></li><li><span><a href="#Dependant-variable" data-toc-modified-id="Dependant-variable-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Dependant variable</a></span></li><li><span><a href="#Splitting-to-train-and-test-data" data-toc-modified-id="Splitting-to-train-and-test-data-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Splitting to train and test data</a></span></li><li><span><a href="#Count-vectorizer" data-toc-modified-id="Count-vectorizer-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Count vectorizer</a></span></li></ul></li><li><span><a href="#Fitting-the-first-iteration" data-toc-modified-id="Fitting-the-first-iteration-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Fitting the first iteration</a></span></li><li><span><a href="#TFIDF" data-toc-modified-id="TFIDF-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>TFIDF</a></span><ul class="toc-item"><li><span><a href="#Lasso-regression-on-TFIDF" data-toc-modified-id="Lasso-regression-on-TFIDF-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Lasso regression on TFIDF</a></span></li></ul></li><li><span><a href="#Random-forest-regressor" data-toc-modified-id="Random-forest-regressor-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Random forest regressor</a></span></li><li><span><a href="#XGBoost-regression" data-toc-modified-id="XGBoost-regression-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>XGBoost regression</a></span></li><li><span><a href="#What-if-I-took-it-as-a-classification?" data-toc-modified-id="What-if-I-took-it-as-a-classification?-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>What if I took it as a classification?</a></span><ul class="toc-item"><li><span><a href="#Model-evaluation" data-toc-modified-id="Model-evaluation-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Model evaluation</a></span></li><li><span><a href="#Model-interpretation" data-toc-modified-id="Model-interpretation-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Model interpretation</a></span></li></ul></li></ul></div>

The aim for question 1 was to predict salaries for jobs based off the descriptions given

In [1]:
#Importing relevant libraries for this section
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import scipy as sp
from sklearn.model_selection import cross_val_score, train_test_split, cross_val_predict, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, LinearRegression, LassoCV, Lasso
from sklearn import metrics
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.externals.six import StringIO  
from nltk.stem import LancasterStemmer, WordNetLemmatizer,PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud

import pydotplus
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import operator
from xgboost import XGBRegressor

In [2]:
#Loading the correct csv into pandas
jobs=pd.read_csv('model_ready.csv',index_col=0)

In [3]:
#Quick visual check of the data
jobs.head()

,description,title,revised_salary,rel_title,ranking,category,salary_level,state,city,cleaned_descriptions
0,Digital Sales Business Analyst\r\r\r\nOpportun...,Digital Business Analyst,105000.0,Business Analyst,Normal,Business AnalystNormal,med,NSW,Sydney,Digital Sales Business Analyst\r\r\r\nOpportun...
1,About the role : As a Business Analyst you wou...,Business Analyst,102500.0,Business Analyst,Normal,Business AnalystNormal,med,NSW,Sydney,About the role : As a Business Analyst you wou...
2,TNT Express is one of the world's leading prov...,Business Analyst,105000.0,Business Analyst,Normal,Business AnalystNormal,med,NSW,Mascot,TNT Express is one of the world's leading prov...
3,For over 15 years Quantium have combined the b...,Agile Business Analyst,105000.0,Business Analyst,Normal,Business AnalystNormal,med,NSW,Sydney,For over 15 years Quantium have combined the b...
4,A vacancy exists in the above unit for a highl...,Business Analyst,105000.0,Business Analyst,Normal,Business AnalystNormal,med,NSW,Darlinghurst,A vacancy exists in the above unit for a highl...


## Choosing the dependant and independant variables

### Independant variables
The independant variables I will be using to make my models will be the descriptions, states and cities. I did not chose to include relative titles or categories because I used that information to compute what the salaries should be so including them would artificailly improve my model due to their high correlation.

#### Coding dummy variables
I used pandas to encode dummy variables for the cities and states for the jobs so I can use them for moddeling

In [4]:
df_dummies = pd.get_dummies(jobs[['city','state']], drop_first=True)
df_dummies.shape

(1315, 98)

Using the dummy variables I encoded the only other variable to use as an independant variable is the job descriptions.

In [5]:
#I created a new dataframe concatenating the independant variables I have chosen
X_df = pd.concat([df_dummies, jobs.cleaned_descriptions], axis=1)

### Dependant variable
I decided to go with tackling the problem in terms of regression although it is not perfect and I anticipate won't yeild very good results. The reason I still decided to do this however is to see how well using text for regression analytics might fair as opposed the calssification.

In [6]:
y = jobs.revised_salary

### Splitting to train and test data

Here I split my data into a train and test set. I will be training all my models on the training set and testing on the test set. This will give me the best idea of how well generalized my model is. I opted with a training set size of 70%.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y, random_state=1,train_size=0.7)

### Count vectorizer
I started my first model using count vectorizer with an n-gram range of 2-3 because I felt this would give me the best results in terms of correctly identifying important phrases. I also used stop words from the NLTK library and limited the max features to 100.

In [8]:
vect = CountVectorizer(ngram_range=(2,3),stop_words='english',max_features=100)
X_train_dtm1 = vect.fit_transform(X_train.cleaned_descriptions)
X_test_dtm1 = vect.transform(X_test.cleaned_descriptions)

To get the result into a working state for my model, I had to transform it from a sparse matrix back to a dense one wit hthe proper headers denoting each phrase.

In [9]:
train_count_vect_df = pd.DataFrame(X_train_dtm1.todense(), columns=vect.get_feature_names())
test_count_vect_df = pd.DataFrame(X_test_dtm1.todense(), columns=vect.get_feature_names())

This allows me to then concatinate the vectorised description with the dummies I created earlier into one dataframe.

In [10]:
X_train_dtm=pd.concat([X_train.drop('cleaned_descriptions',axis=1).reset_index(drop=True), train_count_vect_df], axis=1)
X_test_dtm=pd.concat([X_test.drop('cleaned_descriptions',axis=1).reset_index(drop=True), test_count_vect_df], axis=1)

## Fitting the first iteration

I decided to fit the regression model using Lasso regression right off the bat as I believe it is necessary to reduce the number of independant variables which Lasso will do nicely. Using cross validation, I found the optimal alpha and then fit a model using it onto my data.

In [11]:
optimal_lasso = LassoCV(cv=4).fit(X_train_dtm,y_train)

lasso = Lasso(alpha=optimal_lasso.alpha_)

lasso_scores = cross_val_score(lasso,X_train_dtm,y_train, cv=4)
print('Optimal alpha:',optimal_lasso.alpha_)
print('Mean lasso CV R2:',np.mean(lasso_scores))

Optimal alpha: 473.605101764796
Mean lasso CV R2: 0.19202148206025094


In [12]:
lasso.fit(X_train_dtm,y_train)
y_pred=lasso.predict(X_test_dtm)
print('Lasso train score:',lasso.score(X_train_dtm,y_train))
print('Lasso test score:',lasso.score(X_test_dtm,y_test))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test,y_pred)));

Lasso train score: 0.24311780530042626
Lasso test score: 0.12028277998648207
RMSE: 29753.824782576725


The model created above does not perform particularly well for predicting the salary. This is evident with the low train and test R-squared scores. In terms of how wrong the predictions are, on average the predictions made with this model are off by __$29,753__. The main issue here is that I am taking this as a regression problem and assuming a linear relationship but the way I have imputed the salaries means it will not be strictly linear.

## TFIDF

My next task is to break down the job descriptions like I did during exploratory data analysis. From doing the EDA, it is clear the best results were from using TFIDF which gave a much more appropriate weightage to words for my dataset.I also decided to stem and lemmatize the descriptions to further improve results.

In [13]:
# I needed to create 2 functions from which I will both use to stem and lemmatize words
def lemmatize_word(words):
    lemma=WordNetLemmatizer()
    words = word_tokenize(words)
    lemma_list=[]
    for w in words:
        lemma_list.append(lemma.lemmatize(w))
    return (" ".join(lemma_list))

def stem_word(words):
    stem=PorterStemmer()
    words = word_tokenize(words)
    stem_list=[]
    for w in words:
        stem_list.append(stem.stem(w))
    return (" ".join(stem_list))

def stem_and_lemma(words):
    new_article=lemmatize_word(words)
    new_article=stem_word(words)
    return new_article

Because now I am instead using a TFIDF vectorizer over a count vectorizer, I need to redo the steps above but simply change the vectorizer used to obtain my new independant variables.

In [14]:
jobs['stem_lem']=jobs.cleaned_descriptions.apply(lambda x : stem_and_lemma(x))

In [15]:
X_df = pd.concat([df_dummies, jobs.stem_lem], axis=1)
y = jobs.revised_salary

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y, random_state=1,train_size=0.7)

In [17]:
vect = TfidfVectorizer(ngram_range=(2,3),stop_words='english',max_features=100)
X_train_dtm1 = vect.fit_transform(X_train.stem_lem)
X_test_dtm1 = vect.transform(X_test.stem_lem)

In [18]:
train_count_vect_df = pd.DataFrame(X_train_dtm1.todense(), columns=vect.get_feature_names())
test_count_vect_df = pd.DataFrame(X_test_dtm1.todense(), columns=vect.get_feature_names())

In [19]:
X_train_dtm=pd.concat([X_train.drop('stem_lem',axis=1).reset_index(drop=True), train_count_vect_df], axis=1)
X_test_dtm=pd.concat([X_test.drop('stem_lem',axis=1).reset_index(drop=True), test_count_vect_df], axis=1)

### Lasso regression on TFIDF
I decided to use Lasso regression again here to really showcase the difference between the two vectorizers. Using TFIDF I definitely get a better R squared score but I also get a better RMSE.

In [20]:
optimal_lasso = LassoCV(cv=4).fit(X_train_dtm,y_train)

lasso = Lasso(alpha=optimal_lasso.alpha_)

lasso_scores = cross_val_score(lasso,X_train_dtm,y_train, cv=4)
print('Optimal alpha:',optimal_lasso.alpha_)
print('Mean lasso CV R2:',np.mean(lasso_scores))

Optimal alpha: 85.20377010304789
Mean lasso CV R2: 0.2307490034997837


In [21]:
lasso.fit(X_train_dtm,y_train)
y_pred=lasso.predict(X_test_dtm)
print('Lasso train score:',lasso.score(X_train_dtm,y_train))
print('Lasso test score:',lasso.score(X_test_dtm,y_test))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test,y_pred)));

Lasso train score: 0.31332890180632267
Lasso test score: 0.15479666738896813
RMSE: 29164.320012346598


## Random forest regressor
Using the TFIDF vectorizer, I fit a random forest regression on the data. A random forest model is basically made up of many decision trees, in this case I am using 1000. It then leverages their strong learning property to take the most popular prediction when the model is used.

In [22]:
rfr=RandomForestRegressor(n_estimators=1000)
rfr.fit(X_train_dtm,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [23]:
rfr_scores = cross_val_score(rfr, X_train_dtm, y_train, cv=4)
print('Mean random forest CV R2:',np.mean(rfr_scores))

Mean random forest CV R2: 0.21928349557544752


**There is a significant improvement on RMSE**

This shows me that the random forest is much better at predicting the salaries given the variables I have chosen. If the focus was jsut on predicting salaries this would definitely take precedent over my Lasso regression.

In [24]:
y_pred=rfr.predict(X_test_dtm)
print('Random forest train score:',rfr.score(X_train_dtm,y_train))
print('Random forest test score:',rfr.score(X_test_dtm,y_test))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test,y_pred)));

Random forest train score: 0.9032714218326503
Random forest test score: 0.23701563063879216
RMSE: 27709.52461573283


## XGBoost regression
Seeing the promising results of ensemble modeling, I decided to fit another ensemble model. This time I went with XGBoost. While Random forest takes advantage of the strong learners that individual decision trees can be, XGBoost leverages their ability to be weak learners also. Using a series of weak learners and placing weightage on their mistakes every itteration the theory is we will have a much better prediction model.

In this situation, it did not perform any better than the random forest model I used before.

In [25]:
xgboost=XGBRegressor(n_estimators=1000)
xgboost.fit(X_train_dtm,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [26]:
xgboost_scores = cross_val_score(xgboost, X_train_dtm, y_train, cv=4)
print('Mean random forest CV R2:',np.mean(xgboost_scores))

Mean random forest CV R2: 0.16320533570774864


In [27]:
y_pred=xgboost.predict(X_test_dtm)
print('Random forest train score:',xgboost.score(X_train_dtm,y_train))
print('Random forest test score:',xgboost.score(X_test_dtm,y_test))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test,y_pred)));

Random forest train score: 0.9710269244429689
Random forest test score: 0.22555383634470572
RMSE: 27916.87943475195


## What if I took it as a classification?
I decided to fit a rudimentary model taking it as a classification problem which would fit the dataset much better. This is because of the way I imputed the salaries. Because there were so many NaN values for the salary, by imputing salaries like I did it essentially lent it self to being more  of a classification problem than a regression problem. The model I will fit is a random forest simply because it performed best when tackling the problem as a regression one.

In [28]:
#Baseline = 64.7%
y = jobs.salary_level
y.value_counts(normalize=True)

med     0.647148
low     0.180228
high    0.172624
Name: salary_level, dtype: float64

In [29]:
X_df = pd.concat([df_dummies, jobs.stem_lem], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X_df, y, random_state=1,train_size=0.7)
vect = TfidfVectorizer(ngram_range=(2,3),stop_words='english',max_features=100)
X_train_dtm1 = vect.fit_transform(X_train.stem_lem)
X_test_dtm1 = vect.transform(X_test.stem_lem)
train_count_vect_df = pd.DataFrame(X_train_dtm1.todense(), columns=vect.get_feature_names())
test_count_vect_df = pd.DataFrame(X_test_dtm1.todense(), columns=vect.get_feature_names())
X_train_dtm=pd.concat([X_train.drop('stem_lem',axis=1).reset_index(drop=True), train_count_vect_df], axis=1)
X_test_dtm=pd.concat([X_test.drop('stem_lem',axis=1).reset_index(drop=True), test_count_vect_df], axis=1)
classifier = RandomForestClassifier()

classifier.fit(X_train_dtm, y_train)
classifier_cross = cross_val_score(classifier, X_train_dtm, y_train, cv=4)
print('Mean L1 penatly logistic regression CV score:',np.mean(classifier_cross));

Mean L1 penatly logistic regression CV score: 0.8075602791563891


### Model evaluation
The model I fit performs better than the baseline which is extremely promising. It also seems to be quite consistent in predicting across all three bands of pay that I set out.

In [30]:
y_pred=classifier.predict(X_test_dtm)
classifier_v1=metrics.classification_report(y_test, y_pred)
print(classifier_v1)

             precision    recall  f1-score   support

       high       0.66      0.61      0.63        61
        low       0.75      0.68      0.71        66
        med       0.85      0.88      0.86       268

avg / total       0.80      0.81      0.80       395



### Model interpretation
From the model it is clear that they are placing a lot of importance on a few key words that were actually used to define the salary level. In this case this is definitely not the goal for the classifier. On later itterations it is clear that I should infact add these titles to the stop words list so that they do not get used when doing the prediction and artifically improving the result.

In [31]:
# Get numerical feature importances
importances = list(classifier.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(X_test_dtm.columns, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances[:65]];

Variable: data analyst         Importance: 0.12
Variable: senior busi          Importance: 0.11
Variable: busi analyst         Importance: 0.06
Variable: senior busi analyst  Importance: 0.04
Variable: commun skill         Importance: 0.02
Variable: data analysi         Importance: 0.02
Variable: city_Sydney          Importance: 0.01
Variable: state_NSW            Importance: 0.01
Variable: state_VIC            Importance: 0.01
Variable: abil work            Importance: 0.01
Variable: analyst join         Importance: 0.01
Variable: analyst work         Importance: 0.01
Variable: appli thi            Importance: 0.01
Variable: best practic         Importance: 0.01
Variable: big data             Importance: 0.01
Variable: busi analysi         Importance: 0.01
Variable: busi need            Importance: 0.01
Variable: busi process         Importance: 0.01
Variable: busi requir          Importance: 0.01
Variable: busi stakehold       Importance: 0.01
Variable: busi unit            Importanc

# Summary
For this question as a regression problem I was not able to do very accurate predictions. The lowest RMSE I could acheive with my models was 27000 which means that on average my errors were rather large considering the salaries I used ranged between $19000 and $120000.

Upon changing my approach to that of classification to better suit the data, I was able to obtain quite a decent preliminary model as seen with the f1 score. I can definitely see room for improvement however in this model because it is clear that I can better utilize the stopwords to get a much more generalized model that could be used in the real world where not all job descriptions will have a relevant title within the description potentially.